In [73]:
import sys

In [74]:
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-xunecznu
  Running command git clone -q https://github.com/buguroo/pyknow/ /tmp/pip-req-build-xunecznu


In [75]:
from pyknow import *

YES = 'Да'
NO = 'Нет'

ACTION_TURN_AROUND = 'Развернуться'
ACTION_CONTINUE_DRIVING_STRAIGHT = 'Продолжать движение вперёд по правилам'
ACTION_CONTINUE_DRIVING_CROSSROAD = 'Следуем правилам проезда на перекрёстке'
ACTION_END = 'Мы доехали'
ACTION_STRAIGHT_CROSSROAD_TURN = 'Ехать до ближайшего перекрёстка и повернуть в направлении пункта назначения'
ACTION_CROSSROAD_TURN = 'Развернуться на ближайшем перекрёстке'

class Navigator(KnowledgeEngine):
    def declare_facts(self, facts):
        for fact in facts:
            self.declare(fact)

    @Rule(AND(Fact(navigator_direction = 'Прямо'),
              Fact(forbidden_sign = NO)))
    def forbidden_false(self):
        self.declare(Fact(final_action = ACTION_CONTINUE_DRIVING_STRAIGHT))

    @Rule(OR(Fact(navigator_direction = 'Направо'),
             Fact(navigator_direction = 'Налево')))
    def navigator_direction_right_left(self):
        self.declare(Fact(final_action = ACTION_CONTINUE_DRIVING_CROSSROAD))

    @Rule(Fact(navigator_turn_around = NO))
    def navigator_turn_around_false(self):
        self.declare(Fact(final_action = ACTION_END))

    @Rule(AND(Fact(navigator_turn_around = YES), 
              Fact(ban_on_turning = YES),
              Fact(one_way_traffic = YES)))
    def all_right_turn(self):
        self.declare(Fact(final_action = ACTION_STRAIGHT_CROSSROAD_TURN))

    @Rule(OR(
          AND(Fact(navigator_turn_around = YES),
              Fact(ban_on_turning = YES),
              Fact(one_way_traffic = NO)),
          AND(Fact(navigator_turn_around = YES),
              Fact(ban_on_turning = NO),
              Fact(stop_15_meters = YES))))
    def crossroad_turn(self):
        self.declare(Fact(final_action = ACTION_CROSSROAD_TURN))

    @Rule(OR(
          AND(Fact(navigator_direction = 'Прямо'), 
              Fact(forbidden_sign = YES)),
          AND(Fact(navigator_turn_around = YES),
              Fact(ban_on_turning = NO),
              Fact(stop_15_meters = NO))))
    def turn_true(self):
        self.declare(Fact(final_action = ACTION_TURN_AROUND))

    def factz(self,l):
        for x in l:
            self.declare(x)

    @Rule(Fact(final_action = MATCH.action), salience = 1)
    def what_to_do(self, action):
        print(action)

In [76]:
test = Navigator()

In [77]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Прямо'),
    Fact(forbidden_sign = 'Да')
])

test.run()

Развернуться


FactList([(0, InitialFact()),
          (1, Fact(navigator_direction='Прямо')),
          (2, Fact(forbidden_sign='Да')),
          (3, Fact(final_action='Развернуться'))])

In [78]:
test.reset()
test.factz([
    Fact(navigator_turn_around = YES),
    Fact(ban_on_turning = YES),
    Fact(one_way_traffic = NO)
])

test.run()

Развернуться на ближайшем перекрёстке


In [79]:
test.reset()
test.factz([
    Fact(navigator_turn_around = 'Да'),
    Fact(ban_on_turning = 'Да'),
    Fact(one_way_traffic = 'Нет')
])
test.run()

Развернуться на ближайшем перекрёстке


In [80]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Прямо'),
    Fact(forbidden_sign = 'Нет')
])

test.run()

Продолжать движение вперёд по правилам


In [81]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Направо')
])

test.run()

Следуем правилам проезда на перекрёстке


In [82]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Направо')
])

test.run()

Следуем правилам проезда на перекрёстке


In [83]:

test.reset()
test.factz([
    Fact(navigator_turn_around = 'Нет')
])

test.run()

Мы доехали


In [84]:
test.reset()
test.factz([
    Fact(navigator_turn_around = 'Да'),
    Fact(ban_on_turning = 'Да'),
    Fact(one_way_traffic = 'Да')
])

test.run()

Ехать до ближайшего перекрёстка и повернуть в направлении пункта назначения
